In [4]:
import os
import math
import numpy as np
import pandas as pd
import rasterio as rio
from rasterio.warp import reproject, transform_bounds, Resampling
from rasterio.windows import from_bounds
from rasterio.transform import Affine

In [5]:
# Set directory and file names
data_dir = r'/Users/yasuyukiakita/Data'
results_dir = r'/Users/yasuyukiakita/Projects/python_gis_py312/results'
landfire_dir = os.path.join(data_dir, 'landfire')
lf_2023_fccs_raster = os.path.join(landfire_dir, 'LF2023_FCCS_240_CONUS', 
                                   'Tif', 'LC23_FCCS_240.tif')

# Extract raster using envelope

In [6]:
# Envelope coordinates (longitude/latitude or projected units)
xmin = -2362390
xmax = -1575139
ymin = 1200000
ymax = 2472128

In [7]:
def compute_adjusted_bounds(src_raster, xmin, ymin, xmax, ymax):
    # Read source raster
    with rio.open(src_raster) as src:
        src_crs = src.crs
        src_pixel_x, src_pixel_y = src.res
        src_bounds = src.bounds

    # Check xmin, xmax, ymin, ymax
    if xmin < src_bounds[0]:
        raise ValueError('xmin is smaller than xmin of source extent')
    if xmax > src_bounds[2]:
        raise ValueError('xmax is larger than xmax of source extent')
    if ymin < src_bounds[1]:
        raise ValueError('ymin is smaller than ymin of source extent')
    if ymax > src_bounds[3]:
        raise ValueError('ymax is larger than ymax of source extent')

    # Compute adjusted bounds
    adj_xmin = math.floor((xmin - src_bounds[0]) / src_pixel_x) * src_pixel_x + src_bounds[0]
    adj_xmax = math.ceil((xmax - src_bounds[0]) / src_pixel_x) * src_pixel_x + src_bounds[0]
    adj_ymin = math.floor((ymin - src_bounds[1]) / src_pixel_y) * src_pixel_y + src_bounds[1]
    adj_ymax = math.ceil((ymax - src_bounds[1]) / src_pixel_y) * src_pixel_y + src_bounds[1]

    # Check adjusted xmin, xmax, ymin, ymax
    if adj_xmin > xmin:
        raise ValueError('adjusted xmin is larger than original xmin')
    if adj_xmax < xmax:
        raise ValueError('adjusted xmax is smaller than original xmax')
    if adj_ymin > ymin:
        raise ValueError('adjusted ymin is larger than original ymin')
    if adj_ymax < ymax:
        raise ValueError('adjusted ymax is smaller than original ymax')
    
    return [adj_xmin, adj_ymin, adj_xmax, adj_ymax]

In [8]:
compute_adjusted_bounds(lf_2023_fccs_raster, xmin, ymin, xmax, ymax)

[-2362395.0, 1199985.0, -1575135.0, 2472135.0]

In [9]:
def extract_raster_by_envelope(src_raster, dst_raster, xmin, ymin, xmax, ymax):
    # Adjuste xmin, xmax, ymin, ymax
    adj_xmin, adj_ymin, adj_xmax, adj_ymax = compute_adjusted_bounds(src_raster, xmin, ymin, xmax, ymax)
    # Open the source raster
    with rio.open(src_raster) as src:
        # Create a window from the envelope bounds
        window = from_bounds(adj_xmin, adj_ymin, adj_xmax, adj_ymax, 
                             transform=src.transform)
        
        # Read the data within the window
        data = src.read(window=window)
        
        # Calculate the transform for the new window
        transform = src.window_transform(window)
        
        # Update metadata for output
        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": window.height,
            "width": window.width,
            "transform": transform, 
            "compress": 'lzw', 
        })
        
        # Write the cropped raster to a new file
        with rio.open(dst_raster, "w", **out_meta) as dest:
            dest.write(data)    

In [11]:
dst_raster = os.path.join(results_dir, 'landfire', 'LC23_FCCS_240_CA_only.tif')
extract_raster_by_envelope(lf_2023_fccs_raster, dst_raster, xmin, ymin, xmax, ymax)